## **7. Encrypted Deep Learning**

### **Encrypted computation in PySyft**

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0701 15:24:14.065461 139722714326912 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0701 15:24:14.090709 139722714326912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [0]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [0]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [0]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [0]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

### **Building an Encrypted Database**

In [0]:
import string
import torch as th

In [0]:
char2index = {}
index2char = {}

In [0]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
  char2index[char] = i
  index2char[i] = char

In [0]:
  str_input = "Hello"
  max_len = 8
  

In [0]:
def string2values(str_input, max_len=8):

  str_input = str_input[:max_len].lower()
  
  if(len(str_input)<max_len):
    str_input = str_input + " , " * (max_len-len(str_input))
    
  values = list()
  for char in str_input:
    values.append(char2index[char])
    
  return th.tensor(values).long()


def values2string(input_values):
  s = ""
  for value in input_values:
    s += int2char[int(value)]
  return s

In [0]:
string2values(str_input)

tensor([ 8,  5, 12, 12, 15,  0, 48,  0,  0, 48,  0,  0, 48,  0])

In [0]:
def one_hot(index, length):
  vect = th.zeros(length).long()
  vect[index] = 1
  return vect

In [0]:
one_hot(3,5)

tensor([0, 0, 0, 1, 0])

In [0]:
def string2one_hot_matrix(str_input, max_len=8):

  str_input = str_input[:max_len].lower()
  
  if(len(str_input)<max_len):
    str_input = str_input + " , " * (max_len-len(str_input))
    
  char_vectors = list()
  for char in str_input:
    char_v = one_hot(char2index[char], len(index2char)).unsqueeze(0)
    char_vectors.append(char_v)
    
  return th.cat(char_vectors, dim=0)

In [0]:
matrix = string2one_hot_matrix("Hello")
matrix

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

We can check if two encrypted keys match by doing the following:

In [0]:
str_a = string2one_hot_matrix("Helll")
str_b = string2one_hot_matrix("Hello")

In [0]:
# returns 1 when both strings are the same
def strings_equal(str_a, str_b):

  if len(str_a) == len(str_b):
    vect = (str_a * str_b).sum(1)

    x = vect[0]

    for i in range(vect.shape[0]-1):
      x = x * vect[i+1]
  else:
    x = th.tensor(0)

  return x

strings_equal(str_a,str_b)

tensor(0)

In [0]:
class EncryptedDB():
  
  def __init__(self, *owners, max_key_len=8,max_val_len=8):
    self.max_key_len = max_key_len
    self.max_val_len = max_val_len
  
    # We'll encode the keys with one_hot_matrix and the data with string2values.
    self.keys = list()
    self.values = list()
    self.owners = owners

    
    
  def add_entry(self, key, value):
    key = string2one_hot_matrix(key)
    key = key.share(*self.owners)
    self.values.append(key)
    
    value = string2values(value, max_len=self.max_val_len)
    value = value.share(*self.owners)
    self.values.append(value)
        
    
  
  def query(self, query_str):
    query_matrix = string2one_hot_matrix(query_str)
    query_matrix = query_matrix.share(*self.owners)

    key_matches = list()
    
    for key in self.keys:
      
      key_match = strings_equal(key, query_matrix)
      key_matches.append(key_match)

    result = self.values[0] * key_matches[0]

    for i in range(len(self.values)-1):
      result += self.values[i+1] + key_matches[i+1]
      
    result = result.get()

    return values2string(result).replace(".","")

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
db = EncryptedDB(bob,alice,secure_worker,max_val_len=256)

db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

db.query("key1")

### **Encrypted Deep Learning in PyTorch**

First we'll train a regular model

In [0]:
from torch import nn, optim
import torch.nn.functional as F

#Toy dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(2,20)
    self.fc2 = nn.Linear(20,1)
    
  def forward(self,x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x
  
model =  Net()

def train():
  
  opt = optim.SGD(params=model.parameters(),lr=0.1)
  for i in range(20):
    
    opt.zero_grad()
    pred = model(data)
    loss = ((pred-target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.data)
    
    
train()

tensor(1.5224)
tensor(5.4030)
tensor(19.5920)
tensor(3.5385)
tensor(0.9767)
tensor(0.8976)
tensor(0.8505)
tensor(0.7986)
tensor(0.7350)
tensor(0.6619)
tensor(0.5810)
tensor(0.4948)
tensor(0.4105)
tensor(0.3396)
tensor(0.2667)
tensor(0.2072)
tensor(0.1620)
tensor(0.1242)
tensor(0.0963)
tensor(0.0758)


In [0]:
model(data)
# it's suposed to predict 0,0,1,1

tensor([[ 0.1787],
        [-0.0091],
        [ 0.9044],
        [ 0.8731]], grad_fn=<AddmmBackward>)

Now, we'll encrypt both model and data.

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
#just checking that it is indeed encrypted
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:61526291620 -> alice:99966357011]
 	-> (Wrapper)>[PointerTensor | me:91775101613 -> bob:99043759602]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:81270836623 -> alice:70833606441]
 	-> (Wrapper)>[PointerTensor | me:15704866925 -> bob:84054783600]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:89406463235 -> alice:11130594071]
 	-> (Wrapper)>[PointerTensor | me:73475118996 -> bob:74594146332]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:71261363168 -> alice:38937786591]
 	-> (Wrapper)>[PointerTensor | me:21461818858 -> bob:93208156

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:30947426895 -> alice:92302330790]
	-> (Wrapper)>[PointerTensor | me:79820632789 -> bob:9951067081]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
encrypted_prediction.get().float_precision()

tensor([[ 0.1800],
        [-0.0080],
        [ 0.9050],
        [ 0.8730]])

### **Encrypted Deep Learning in Keras**

Now we'll reproduce the previous section in Keras

In [0]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3,3), input_shape=input_shape))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3)))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])

model.fit(x_train, y_train, batch_size = batch_size,
          epochs=epochs, verbose=1, validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss', score[0])
print('Test accuracy', score[1])

11493376/11490434 [==============================] - 0s 0us/step


W0628 14:20:18.470101 140035515029376 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 108us/sample - loss: 2.3030 - acc: 0.1044 - val_loss: 2.3024 - val_acc: 0.1038
Epoch 2/2
60000/60000 [==============================] - 4s 60us/sample - loss: 2.3015 - acc: 0.1095 - val_loss: 2.3009 - val_acc: 0.1116
Test loss 2.30091893081665
Test accuracy 0.1116


In [0]:
model.save('short-conv-mnist.h5')

This time we'll do a keras hook.

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

In [0]:
# Model
num_classes = 10
input_shape = (1,28,28,1)

model = Sequential()

model.add(Conv2D(10, (3,3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3)))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation="softmax"))

# loading pre-trained weights
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

In [0]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

In [0]:
model.share(alice, bob, carol)

In [0]:
model.serve(num_requests=3)

And that should be enough to start our server. If you want to check how the clients work, go to *https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/* and click on Part13 to follow the full tutorial.